In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os

# Configurando o WebDriver
driver = webdriver.Chrome()

try:
    url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'
    driver.get(url)

    # Espera a tabela carregar
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'table'))
    )

    # Função para obter a data do pregão
    def obter_data_pregao():
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            span = soup.find('span', class_='strong')
            if span:
                data_texto = span.text.strip()
                return datetime.strptime(data_texto, "%d/%m/%Y").strftime('%Y-%m-%d')
        except Exception as e:
            print(f"Erro ao obter a data do pregão: {e}")
        return datetime.now().strftime('%Y-%m-%d')

    data_pregao = obter_data_pregao()
    print(f"Data do pregão detectada: {data_pregao}")

    pasta_saida = f'data_pregao={data_pregao}'
    os.makedirs(pasta_saida, exist_ok=True)

    def extrair_dados(html):
        soup = BeautifulSoup(html, 'html.parser')
        tabela = soup.find('table', class_='table')
        dados = []

        if tabela:
            linhas = tabela.find_all('tr')
            for tr in linhas:
                tds = tr.find_all('td')
                linha = [td.get_text(strip=True) for td in tds]
                if len(linha) == 5:
                    dados.append(linha)
        return dados

    def extrair_linhas_finais(html):
        soup = BeautifulSoup(html, 'html.parser')
        linhas_finais = []

        linhas = soup.select('tr:has(td[colspan="3"])')
        for linha in linhas:
            colunas = linha.find_all('td')
            linha_dados = [td.get_text(strip=True) for td in colunas]
            if 'Quantidade Teórica Total' in linha_dados[0]:
                linhas_finais.append([linha_dados[0], '', '', linha_dados[1], linha_dados[2]])
            elif 'Redutor' in linha_dados[0]:
                linhas_finais.append([linha_dados[0], '', '', linha_dados[1], ''])
        return linhas_finais

    dados = []

    while True:
        html = driver.page_source
        dados.extend(extrair_dados(html))

        try:
            proximo = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'li.pagination-next a'))
            )

            pai_classe = proximo.find_element(By.XPATH, '..').get_attribute('class')
            if 'disabled' in pai_classe:
                break

            ActionChains(driver).move_to_element(proximo).click().perform()

            WebDriverWait(driver, 10).until(EC.staleness_of(proximo))
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'table'))
            )

        except (NoSuchElementException, TimeoutException):
            break

    # Extraindo dados finais adicionais
    dados.extend(extrair_linhas_finais(driver.page_source))

    for linha in dados:
        print(linha)

    if dados:
        colunas = ['Código', 'Nome', 'Tipo', 'Qtde. Teórica', 'Part. (%)']
        df = pd.DataFrame(dados, columns=colunas)

        df['Qtde. Teórica'] = df['Qtde. Teórica'].str.replace('.', '', regex=True)
        df['Qtde. Teórica'] = pd.to_numeric(df['Qtde. Teórica'], errors='coerce')

        df['Part. (%)'] = df['Part. (%)'].str.replace(',', '.', regex=True)
        df['Part. (%)'] = pd.to_numeric(df['Part. (%)'], errors='coerce')

        df['data pregão'] = pd.to_datetime(data_pregao)

        caminho_saida = f'{pasta_saida}/dados_{data_pregao}.parquet'
        df.to_parquet(caminho_saida, index=False)
        print("Dados extraídos e salvos com sucesso!")
    else:
        print("Nenhum dado encontrado.")

finally:
    driver.quit()

Data do pregão detectada: 2025-06-04
['ALOS3', 'ALLOS', 'ON      NM', '476.976.044', '0,489']
['ABEV3', 'AMBEV S/A', 'ON', '4.394.835.131', '2,830']
['ASAI3', 'ASSAI', 'ON      NM', '1.345.897.506', '0,734']
['AURE3', 'AUREN', 'ON      NM', '323.738.747', '0,149']
['AZZA3', 'AZZAS 2154', 'ON      NM', '136.643.320', '0,282']
['B3SA3', 'B3', 'ON      NM', '5.200.055.464', '3,389']
['BBSE3', 'BBSEGURIDADE', 'ON      NM', '637.332.335', '1,074']
['BBDC3', 'BRADESCO', 'ON  EJ  N1', '1.469.064.981', '0,958']
['BBDC4', 'BRADESCO', 'PN  EJ  N1', '5.111.682.020', '3,877']
['BRAP4', 'BRADESPAR', 'PN      N1', '250.982.988', '0,180']
['BBAS3', 'BRASIL', 'ON  EX  NM', '2.842.613.858', '3,003']
['BRKM5', 'BRASKEM', 'PNA     N1', '265.388.400', '0,127']
['BRAV3', 'BRAVA', 'ON      NM', '451.311.960', '0,402']
['BRFS3', 'BRF SA', 'ON      NM', '832.617.717', '0,785']
['BPAC11', 'BTGP BANCO', 'UNT ATZ N2', '1.287.247.964', '2,399']
['CXSE3', 'CAIXA SEGURI', 'ON      NM', '600.000.000', '0,403']
['CMI